In [65]:
import os
import tsfel
import pandas as pd
import numpy as np
from glob import glob


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

# # load dataset
# df = pd.read_csv('train/segment0.csv')

# Retrieves a pre-defined feature configuration file to extract all available features
# cfg = tsfel.get_features_by_domain()

# # Extract features
# X = tsfel.time_series_features_extractor(cfg, df)

In [2]:
cfg = tsfel.get_features_by_domain('statistical')

In [3]:
df_activity = pd.read_csv("activities.csv")

In [4]:
def get_act_id(seg_id):
    seg = df_activity[df_activity["segment_id"]==seg_id]
    activity_id = seg["activity_id"].values
    return int(activity_id)

In [5]:
def handle_missing_values(df):
    df['x']=df['x'].replace(0, np.nan)
    df['y']=df['y'].replace(0, np.nan)
    df['z']=df['z'].replace(0, np.nan)
    return df

In [6]:
def load_data(csv_file):

    y_list = []
    x_data_list = []

    csv_df = pd.read_csv(csv_file)
    csv_df = handle_missing_values(csv_df)
    csv_df.dropna(inplace=True)
    act_id = get_act_id(int(os.path.splitext(os.path.basename(csv_file))[0].replace("segment","")))
    
    return csv_df,act_id

In [7]:
def features_extraction(df):
    X = tsfel.time_series_features_extractor(cfg, df)
    return X

In [8]:
def fill_missing_values(df):
    """ Handle eventual missing data. Strategy: replace with mean.
    
      Parameters
      ----------
      df pandas DataFrame
      Returns
      -------
        Data Frame without missing values.
    """
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(), inplace=True)
    return df

In [11]:
csv_files = glob("train/*")

In [68]:
df = pd.read_csv(csv_files[0])
df_ = features_extraction(df)

*** Feature extraction started ***

*** Feature extraction finished ***


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Using default sampling frequency set in configuration file.
  


In [69]:
df_

,0_ECDF Percentile Count_0,0_ECDF Percentile Count_1,0_ECDF Percentile_0,0_ECDF Percentile_1,0_ECDF Slope,0_ECDF_0,0_ECDF_1,0_ECDF_2,0_ECDF_3,0_ECDF_4,...,2_Max,2_Mean,2_Mean absolute deviation,2_Median,2_Median absolute deviation,2_Min,2_Root mean square,2_Skewness,2_Standard deviation,2_Variance
0,23.0,94.0,-2.068,0.306,0.197785,0.008475,0.016949,0.025424,0.033898,0.042373,...,10.454,1.100966,1.638232,0.081,0.076,0.0,2.75818,2.569853,2.528919,6.395433


In [41]:
X_feature_data_list = []
y_list = []
for csv_file in csv_files:
    x,y = load_data(csv_file)
    df_ = features_extraction(x)
    df_ = fill_missing_values(df_)
    X_feature_data_list.append(df_.values.reshape(-1))
    y_list.append(y)

*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Using default sampling frequency set in configuration file.
  



*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction star

*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finished ***
*** Feature extraction started ***

*** Feature extraction finis

In [42]:
df_feature = pd.DataFrame(X_feature_data_list)

In [43]:
df_feature["activity"] = y_list

In [59]:
df_feature = df_feature.dropna()

In [60]:
# X から NaN を含む列を削除する
df_feature.drop(df_feature.columns[np.isnan(df_feature).any()], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,activity
0,22.0,88.0,-2.106,0.421,0.148368,0.009009,0.018018,0.027027,0.036036,0.045045,...,1.170306,1.715308,0.0900,0.0760,0.001,2.843820,2.462243,2.591852,6.717698,9
1,22.0,90.0,-2.145,0.996,0.163185,0.008850,0.017699,0.026549,0.035398,0.044248,...,1.199549,1.824240,0.0750,0.0640,0.001,2.968996,2.341018,2.715883,7.376020,9
2,22.0,88.0,-2.106,0.919,0.362319,0.009009,0.018018,0.027027,0.036036,0.045045,...,3.249198,3.285901,1.1810,1.2300,-0.218,4.724386,0.351083,3.429656,11.762538,2
3,23.0,95.0,-1.417,0.842,0.724638,0.008403,0.016807,0.025210,0.033613,0.042017,...,2.670798,2.533595,1.5960,1.5390,0.006,3.869314,0.607299,2.799719,7.838424,2
4,23.0,94.0,-2.528,-0.191,0.186567,0.008475,0.016949,0.025424,0.033898,0.042373,...,2.862653,2.940096,0.5870,0.5530,0.004,4.320492,0.682430,3.236027,10.471873,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,67.0,271.0,-2.030,-1.264,0.724638,0.002950,0.005900,0.008850,0.011799,0.014749,...,0.349950,0.325053,0.1710,0.0710,-0.023,0.937030,6.101390,0.869230,0.755560,3
174,21.0,84.0,-4.022,-0.766,0.120831,0.009434,0.018868,0.028302,0.037736,0.047170,...,-7.515255,1.669087,-8.4980,0.0735,-9.336,7.921906,2.313616,2.505502,6.277543,3
175,22.0,90.0,-1.877,0.459,0.435540,0.008850,0.017699,0.026549,0.035398,0.044248,...,5.488283,3.613581,8.2040,0.7880,0.062,6.672377,-0.447257,3.794650,14.399366,3
176,15.0,61.0,-1.608,-0.268,0.343407,0.012987,0.025974,0.038961,0.051948,0.064935,...,1.689584,2.105551,0.2970,0.2310,0.009,3.171536,1.751687,2.684017,7.203947,9


In [61]:
X_train,X_test,y_train,y_test = train_test_split(df_feature.drop("activity",axis=1).values,df_feature["activity"].values,test_size=0.3)

In [62]:
model_ml = RandomForestClassifier(n_estimators=500,n_jobs=-1)

In [63]:
model_ml.fit(X_train,y_train)
y_predict = model_ml.predict(X_test)

In [66]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           2       0.75      0.75      0.75        12
           3       0.69      0.82      0.75        11
           4       1.00      0.50      0.67        10
           6       1.00      0.50      0.67         8
           9       0.75      1.00      0.86         3
          12       0.62      1.00      0.77        10

    accuracy                           0.74        54
   macro avg       0.80      0.76      0.74        54
weighted avg       0.80      0.74      0.73        54



In [70]:
df_.columns

Index(['0_ECDF Percentile Count_0', '0_ECDF Percentile Count_1',
       '0_ECDF Percentile_0', '0_ECDF Percentile_1', '0_ECDF Slope',
       '0_ECDF_0', '0_ECDF_1', '0_ECDF_2', '0_ECDF_3', '0_ECDF_4',
       ...
       '2_Max', '2_Mean', '2_Mean absolute deviation', '2_Median',
       '2_Median absolute deviation', '2_Min', '2_Root mean square',
       '2_Skewness', '2_Standard deviation', '2_Variance'],
      dtype='object', length=111)